In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


############## Settings for Scotese Paleomap
PaleomapDictionary = {}
PaleomapDictionary['name'] = 'Paleomap'
PaleomapDictionary['reconstruction_model'] = Reconstructions.fetch_Scotese()
PaleomapDictionary['raster_sequence'] = Paleogeography.fetch_Paleomap()
PaleomapDictionary['maximum_time'] = 350.
PaleomapDictionary['time_bin_size'] = 5.
PaleomapDictionary['anchor_plate_id'] = 201
PaleomapDictionary['raster_anchor_plate_id'] = 0

#Paleomap = Reconstructions.fetch_Scotese()
#PaleoDEM = Paleogeography.fetch_Paleomap()


############## Settings for Boschman model
boschman_rotation_model = ReconstructionModel('')
boschman_rotation_model.add_rotation_model('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/boschman_reverse_engineered_rotations.rot')
boschman_rotation_model.add_static_polygons('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/reconstructed_0.00Ma.shp')

raster_dict = {}
for reconstruction_time in np.arange(0,81,1):
    raster_dict[reconstruction_time] = '/Users/simon/GIT/bx/andes//boschman/grids/boschman_DEM_{:0.0f}Ma.nc'.format(reconstruction_time)
boschman_rasters = collections.OrderedDict(sorted(raster_dict.items()))


BoschmanDictionary = {}
BoschmanDictionary['name'] = 'Boschman'
BoschmanDictionary['reconstruction_model'] = boschman_rotation_model
BoschmanDictionary['raster_sequence'] = boschman_rasters
BoschmanDictionary['maximum_time'] = 80.
BoschmanDictionary['time_bin_size'] = 5.
BoschmanDictionary['anchor_plate_id'] = 201
BoschmanDictionary['raster_anchor_plate_id'] = 201


########## Geochemistry Inputs
df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv',
                          longitude_field_name='Longitude', latitude_field_name='Latitude')

model_dir = '../luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = joymap.select_orogens(df,gdf=None, 
                           orogen_names='Cordilleran', 
                           continent_names='South America',
                           region=[-100, -50, -60, 20])


pilger_volcanics = joymap.load_pilger_volcanics('/Users/simon/OneDrive/Andes_works//datafiles/2022-2-NGG0Q7_Pilger_Andean-Igneous-Radiometric-Dates.xlsx')

#df.plot()

In [13]:
def make_timeslice_figures(df, reconstruction_time, time_bin_size, space_bin_size, 
                           reconstruction_model, raster_sequence, 
                           anchor_plate_id, raster_anchor_plate_id,
                           gc_interpolator_dict, 
                           calibration, mohometer_selection,
                           region, projection, perspective, 
                           volcanics=None):
    
    
    if isinstance(mohometer_selection, list):
        mohometer_description_string = '|'.join(mohometer_selection)
    else:
        mohometer_description_string = str(mohometer_selection)

    
    fig = pygmt.Figure()

    binned_elevations = joymap.timeslice_plot(df, reconstruction_time,
                                              time_bin_size, space_bin_size, 
                                              fig, reconstruction_model, raster_sequence, 
                                              anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                              gc_interpolator_dict=gc_interpolator_dict, 
                                              calibration=calibration, mohometer_selection=mohometer_selection,
                                              column_marker_size='0.2c',
                                              region=region, projection=projection, perspective=perspective, 
                                              volcanics=volcanics, return_type='binned_elevations')

    joymap.add_labels(fig, reconstruction_time, add_colorbar=True)

    fig.savefig('../images/sequence_{:s}/elevations_plus_volcanism_{:s}_{:s}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                                   calibration,
                                                                                                   mohometer_description_string, 
                                                                                                   reconstruction_time))

    fig = pygmt.Figure()

    binned_residuals = joymap.timeslice_plot(df, reconstruction_time,
                                             time_bin_size, space_bin_size, 
                                             fig, reconstruction_model, raster_sequence, 
                                             anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                             gc_interpolator_dict=gc_interpolator_dict, 
                                             calibration=calibration, mohometer_selection=mohometer_selection,
                                             region=region, projection=projection, perspective=perspective, 
                                             volcanics=None, residuals=True, return_type='binned_residuals')

    joymap.add_labels(fig, reconstruction_time, add_colorbar=True, colorbar_title='Residual Elevation [m]')

    fig.savefig('../images/sequence_{:s}/elevation_residuals_{:s}_{:s}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                   calibration,
                                                                                   mohometer_description_string, 
                                                                                   reconstruction_time))

    joymap.residuals_crossplot(binned_elevations, binned_residuals,
                               fname='../images/sequence_{:s}/crossplot_{:s}_{:s}_{:0.1f}Ma.png'.format(MODEL['name'],
                                                                                                        calibration,
                                                                                                        mohometer_description_string, 
                                                                                                        reconstruction_time))
    #break
    return


In [15]:
from joblib import Parallel, delayed

###########################################
#MODEL = PaleomapDictionary
MODEL = BoschmanDictionary
###########################################

region = [-85, -55, -58, 12]
#region = [-76, -66, -25, -12]
projection="M10c"
perspective = [240, 35]

###########################################



time_bin_size = MODEL['time_bin_size']
space_bin_size = 1.

#calibration = 'FarnerLee'
#mohometer_selection = ['gd_yb_elevation']

#calibration = 'Hu'
#mohometer_selection = ['sr_y_elevation']

#calibration = 'luffi'
#mohometer_selection = 50


plot_calibrations = [
    ('luffi', 41),
    ('luffi', 'la_yb_elevation'),
    ('Hu', 'la_yb_elevation'),
    ('Hu', 'sr_y_elevation'),
    ('FarnerLee', 'la_yb_elevation'),
    ('FarnerLee', 'gd_yb_elevation')
]


anchor_plate_id = MODEL['anchor_plate_id']
raster_anchor_plate_id = MODEL['raster_anchor_plate_id']

reconstruction_model = MODEL['reconstruction_model']
raster_sequence = MODEL['raster_sequence']

num_cpus = 1

for calibration, mohometer_selection in plot_calibrations:
 
    if num_cpus==1:
        
        for reconstruction_time in np.arange(0,MODEL['maximum_time']+time_bin_size,time_bin_size):

            make_timeslice_figures(df, reconstruction_time, time_bin_size, space_bin_size, 
                               reconstruction_model, raster_sequence, 
                               anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                               gc_interpolator_dict=gc_interpolator_dict, 
                               calibration=calibration, mohometer_selection=mohometer_selection,
                               region=region, projection=projection, perspective=perspective, 
                               volcanics=pilger_volcanics)
        
    else:
        Parallel(n_jobs=num_cpus)(delayed(make_timeslice_figures)(
            df, reconstruction_time, time_bin_size, space_bin_size, 
            reconstruction_model, raster_sequence, 
            anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
            gc_interpolator_dict=gc_interpolator_dict, 
            calibration=calibration, mohometer_selection=mohometer_selection,
            region=region, projection=projection, perspective=perspective, volcanics=pilger_volcanics
        ) for reconstruction_time in np.arange(0,MODEL['maximum_time']+time_bin_size,time_bin_size))

        

    


Number of samples after basic filtering 19309
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 14840
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


0.0
Number of samples after basic filtering 19309
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 14840
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


Number of samples after basic filtering 444
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 343
TODO implement min/max elevation cutoffs


...

grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


Number of samples after basic filtering 67
Number of these samples with a valid sio2 = 60
Number of these samples with major element sum > 98%= 21
Final number of samples passed = 21
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


80.0
Number of samples after basic filtering 67
Number of these samples with a valid sio2 = 60
Number of these samples with major element sum > 98%= 21
Final number of samples passed = 21
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


In [5]:
###########################################
#MODEL = PaleomapDictionary
MODEL = BoschmanDictionary
###########################################
region = [-85, -55, -58, 12]
#region = [-76, -66, -25, -12]
projection="M10c"
perspective = [240, 35]

###########################################



time_bin_size = 10
space_bin_size = 1.


plot_calibrations = [
    ('luffi', 41),
    ('luffi', 'la_yb_elevation'),
    ('Hu', 'la_yb_elevation'),
    ('Hu', 'sr_y_elevation'),
    ('FarnerLee', 'la_yb_elevation'),
    ('FarnerLee', 'gd_yb_elevation')
]


anchor_plate_id = MODEL['anchor_plate_id']
raster_anchor_plate_id = MODEL['raster_anchor_plate_id']

reconstruction_model = MODEL['reconstruction_model']
raster_sequence = MODEL['raster_sequence']



for calibration, mohometer_selection in plot_calibrations:

    if isinstance(mohometer_selection, list):
        mohometer_description_string = '|'.join(mohometer_selection)
    else:
        mohometer_description_string = str(mohometer_selection)




    fig = pygmt.Figure()

    for reconstruction_time in np.arange(10,41,time_bin_size): 

        binned_elevations = joymap.timeslice_plot(df, reconstruction_time,
                                                      time_bin_size, space_bin_size, 
                                                      fig, reconstruction_model, raster_sequence, 
                                                      anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                                      gc_interpolator_dict=gc_interpolator_dict, 
                                                      calibration=calibration, mohometer_selection=mohometer_selection,
                                                      column_marker_size='0.2c',
                                                      region=region, projection=projection, perspective=perspective, 
                                                      volcanics=pilger_volcanics, return_type='binned_elevations')

        joymap.add_labels(fig, reconstruction_time, x=2.6, y=0.8, add_colorbar=False)

        fig.shift_origin(yshift='-11c')


    fig.shift_origin(yshift='44c', xshift='32c')

    for reconstruction_time in np.arange(50,81,time_bin_size):
        binned_elevations = joymap.timeslice_plot(df, reconstruction_time,
                                                      time_bin_size, space_bin_size, 
                                                      fig, reconstruction_model, raster_sequence, 
                                                      anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                                      gc_interpolator_dict=gc_interpolator_dict, 
                                                      calibration=calibration, mohometer_selection=mohometer_selection,
                                                      column_marker_size='0.2c',
                                                      region=region, projection=projection, perspective=perspective, 
                                                      volcanics=pilger_volcanics, return_type='binned_elevations')

        joymap.add_labels(fig, reconstruction_time, x=2.6, y=0.8, add_colorbar=False)

        fig.shift_origin(yshift='-11c')


    fig.shift_origin(yshift='8c', xshift='-8c')
    with pygmt.config(FONT_ANNOT_PRIMARY='36p', FONT_LABEL='48p'):
        fig.colorbar(position='JBC+jBC+o0c+w32c/1.6c+h', frame=['x+lElevation [m]'])

    fig.savefig('../images/elevation_vs_{:s}_perspective_{:s}_{:s}.png'.format(MODEL['name'],
                                                                               calibration,
                                                                               mohometer_description_string))
#fig.show(width=1000)


Number of samples after basic filtering 587
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 415
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


10
Number of samples after basic filtering 231
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 139
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


20
Number of samples after basic filtering 409
TODO implement a specific alkaline/subalkaline boundary
Final number of samples passed = 171
TODO implement min/max elevation cutoffs


...

60
Number of samples after basic filtering 119
Number of these samples with a valid sio2 = 107
Number of these samples with major element sum > 98%= 64
Final number of samples passed = 64
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


70
Number of samples after basic filtering 126
Number of these samples with a valid sio2 = 106
Number of these samples with major element sum > 98%= 46
Final number of samples passed = 46
TODO implement min/max elevation cutoffs


grdinfo [WARNING]: Guessing of registration in conflict between x and y, using gridline
grdtrack [WARNING]: Some input points were outside the grid domain(s).


80
